In [1]:
INPUT_FILE = '../data/processed/enriched_df.csv'
POPULATION_FILE = '../data/raw/total_pop_1960_2018.csv'
META_DATA = '../data/raw/meta_data.csv'

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cufflinks as cf
import seaborn as sns
cf.go_offline()

#%matplotlib inline

In [3]:
import plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [62]:
from ipywidgets import interact
import ipywidgets as widgets

In [3]:
%load_ext autoreload
%autoreload 2

## Read in data

In [4]:
df = pd.read_csv(INPUT_FILE)
# df_population = pd.read_csv(POPULATION_FILE, skiprows=4)

In [5]:
df.head()

,country,year,sex,age,suicides_no,population
0,Albania,1985,female,15-24 years,NaN,277900.0
1,Albania,1985,female,25-34 years,NaN,246800.0
2,Albania,1985,female,35-54 years,NaN,267500.0
3,Albania,1985,female,5-14 years,NaN,298300.0
4,Albania,1985,female,55-74 years,NaN,138700.0


## Plot some graphs

In [6]:
df_suicides = df.groupby(['year', 'sex', 'country']).sum().reset_index()

In [7]:
df_suicides.head()

,year,sex,country,suicides_no,population
0,1979,female,Argentina,508.0,12532000.0
1,1979,female,Australia,479.0,6641600.0
2,1979,female,Austria,0.0,3742700.0
3,1979,female,Bahamas,0.0,95800.0
4,1979,female,Barbados,0.0,119800.0


In [8]:
df_suicides['per_100000'] = df_suicides['suicides_no'] / df_suicides['population'] * 100000

In [9]:
df_suicides

,year,sex,country,suicides_no,population,per_100000
0,1979,female,Argentina,508.0,1.253200e+07,4.053623
1,1979,female,Australia,479.0,6.641600e+06,7.212118
2,1979,female,Austria,0.0,3.742700e+06,0.000000
3,1979,female,Bahamas,0.0,9.580000e+04,0.000000
4,1979,female,Barbados,0.0,1.198000e+05,0.000000
...,...,...,...,...,...,...
7291,2016,male,Sweden,784.0,4.620777e+06,16.966843
7292,2016,male,Tajikistan,154.0,4.269218e+06,3.607218
7293,2016,male,Thailand,3283.0,3.000200e+07,10.942602
7294,2016,male,United Kingdom,0.0,3.031980e+07,0.000000


### Make some plots

In [11]:
import pycountry

In [14]:
pycountry.countries.get(name="France").alpha_3

'FRA'

### Add country code

In [20]:
def add_code(country_name):
    try:
        return pycountry.countries.get(name=country_name).alpha_3 
    except AttributeError:
        return ""

In [23]:
df_suicides["code"] = df_suicides["country"].apply(add_code)

In [21]:
add_code("France")

'FRA'

In [10]:
scl = [[0.0, '#ffffff'],[0.2, '#ff9999'],[0.4, '#ff4d4d'], 
       [0.6, '#ff1a1a'],[0.8, '#cc0000'],[1.0, '#4d0000']] # reds

In [24]:
data = [ dict(
            type='choropleth', # type of map-plot
            colorscale = "Reds",
            locations = df_suicides['code'], # the column with the state
            z = df_suicides['per_100000'].astype(float), # the variable I want to color-code
            text = df_suicides['country'], # hover text
            marker = dict(     # for the lines separating states
                        line = dict (
                                  color = 'rgb(255,255,255)', 
                                  width = 2) ),               
            colorbar = dict(
                        title = "Murder rate per 100,000 people")
            ) 
       ]

In [26]:
layout = dict(title="bla", geo=dict(showframe=False, projection={'type': 'natural earth'}))

In [27]:
fig = dict(data=data, layout=layout)

In [28]:
plotly.offline.iplot(fig)

In [ ]:
# https://amaral.northwestern.edu/blog/step-step-how-plot-map-slider-represent-time-evolu
# https://medium.com/@kbrook10/day-8-data-visualization-how-to-use-choropleth-maps-63c0fedfd5e7